In [1]:
from builtins import range
from past.utils import old_div
from collections import namedtuple
import random
import time
import math
import logging
import struct
import socket
import os
import sys
import copy

import gym
import minerl

import torch

import matplotlib.pyplot as plt

from dqnagent import DQNAgent

import faulthandler; faulthandler.enable()

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) # set to INFO if you want fewer messages

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [2]:
env = gym.make('MineRLNavigateDense-v0')
obs = env.reset()

INFO:minerl.env.malmo.instance.9b298d:Starting Minecraft process: ['/tmp/tmp7rm6fg7s/Minecraft/launchClient.sh', '-port', '11650', '-env', '-runDir', '/tmp/tmp7rm6fg7s/Minecraft/run']
INFO:minerl.env.malmo.instance.9b298d:Starting process watcher for process 14361 @ localhost:11650
INFO:minerl.env.malmo.instance.9b298d:Minecraft process ready
INFO:minerl.env.malmo:Logging output of Minecraft to ./logs/mc_2650.log


In [3]:
# act_keys = list(env.action_space.sample().keys())
act_keys = ['attack','back','forward','jump','left','place','right','sneak','sprint','camera']
n_actions = len(act_keys) + 2 #act_keys + camera(the other direction) + no-op

video_height = 64
video_width = 64
    
def im_prep(data):
    # data = torch.tensor(data)
    # data = data.view(video_height,video_width,-1)
    marg = 255/2.0
    data = (data - marg)/marg
    return data.permute(2,0,1).unsqueeze(0)

def format_action(no_act,act_ind): 
    if act_ind > 10:
        act_ind = 10
    elif act_ind < 0:
        act_ind = 0

    action = no_act()

    if act_ind < 9:
        action[act_keys[act_ind]] = 1
    elif act_ind == 9 or act_ind == 10:
        action[act_keys[-1]] = 90*pow(-1,act_ind%9)

    return action

In [5]:
done = False
num_reps = 3

dqn_agent = DQNAgent(video_height,video_width,n_actions,device)

for i_episode in range(num_reps):
    obs = env.reset()
    while not done:
        state = im_prep(obs['pov'])

        # Select and perform an action
        action = dqn_agent.act(state)
        action_command = format_action(action.squeeze())
        # action = env.action_space.sample()

        # Observe new state
        next_obs, reward, done, _ = env.step(action)
        next_state = im_prep(next_obs['pov'])
        reward = torch.tensor([reward], device=device)

        # Store the transition in memory
        dqn_agent.memory.push(state, action, next_state, reward)
        # Move to the next state
        state = next_state
        # state = copy.deepcopy(next_state)

        # Perform one step of the optimization (on the target network)
        dqn_agent.optimize()

    # Update the target network, copying all weights and biases in DQN
    if i_episode % dqn_agent.TARGET_UPDATE == 0:
        dqn_agent.target_update()

    logger.info("Episode has ended.")   

AttributeError: 'numpy.ndarray' object has no attribute 'permute'